1.1.  导入python需要库

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

1. 2.导入数据

In [ ]:
sample = pd.read_csv('/kaggle/input/GiveMeSomeCredit/sampleEntry.csv')
credit_train = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-training.csv')
credit_test = pd.read_csv('/kaggle/input/GiveMeSomeCredit/cs-test.csv')

2.1 数据延申

2.1.1查看数据集构成

In [ ]:
credit_train.info()

2.1.2查看数据类型占比

In [ ]:
credit_train.describe()

2.2 EDA

查看之间的两两关系：

In [ ]:
credit_train.columns

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(credit_train.isnull(),yticklabels=False,cbar=False)

存在缺少数据的情况。

In [ ]:
plt.figure(figsize=(15,10))
sns.jointplot(data = credit_train, x= 'age', y = 'SeriousDlqin2yrs')

In [ ]:
sns.distplot(credit_train['age'].dropna(),kde=False,color='darkred',bins=30)

In [ ]:
sns.distplot(credit_train['DebtRatio'].dropna(),kde=False,color='darkred',bins=500)

In [ ]:
credit_train.corr()

In [ ]:
credit_train.head()

3.1 补充数据

In [ ]:
credit_train['MonthlyIncome'].fillna(credit_train['MonthlyIncome'].mean(),inplace=True)
credit_test['MonthlyIncome'].fillna(credit_test['MonthlyIncome'].mean(),inplace=True)
credit_train['NumberOfDependents'].fillna(credit_train['NumberOfDependents'].mode()[0], inplace=True)
credit_test['NumberOfDependents'].fillna(credit_test['NumberOfDependents'].mode()[0], inplace=True)
credit_test['MonthlyIncome'].fillna(credit_test['MonthlyIncome'].mean(),inplace=True)
plt.figure(figsize=(15,10))
sns.heatmap(credit_train.isnull(),yticklabels=False,cbar=False)

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(credit_train.corr(),annot=True)

检查数据的相关性，得出结论NumberOfTime30-59DaysPastDueNotWorse, NumberOfTimes90DaysLate, NumberOfTime60-89DaysPastDueNotWorse三者相关性为其中较高

In [ ]:
credit_train

In [ ]:
credit_test

In [ ]:
print(credit_train.isnull().sum())

In [ ]:
print(credit_test.isnull().sum())

4.训练模型

In [ ]:
X_train = credit_train[['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents']]
y_train = credit_train['SeriousDlqin2yrs']
X_test = credit_test[['RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents']]
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

In [ ]:
rfc_pred = rfc.predict(X_test)
rfc_proba = rfc.predict_proba(X_test)

In [ ]:
df=pd.DataFrame(rfc_proba,columns=['Id','Probability'])
df.head()

In [ ]:
ind=credit_train['Unnamed: 0']
df['Id']=ind
df.head()

输出训练数据集

In [ ]:
export_csv = df.to_csv('credit_score_random_forest.csv',index = None,header=True)